In [ ]:
! pip install patchify

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from matplotlib import pyplot as plt
import random

In [ ]:
minmaxscaler = MinMaxScaler()

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler

# Set your dataset root folder
dataset_root_folder = '/content/drive/MyDrive/Colab Notebooks/datasets/Aug/'
output_root_folder = '/content/drive/MyDrive/Colab Notebooks/datasets/Aug/'  # Specify the path for saving patched images and masks

# Set the image patch size
image_patch_size = 256  # Change this to your desired patch size

# Create output folders if they don't exist
output_images_folder = os.path.join(output_root_folder, 'patched_images/')
output_masks_folder = os.path.join(output_root_folder, 'patched_masks/')
os.makedirs(output_images_folder, exist_ok=True)
os.makedirs(output_masks_folder, exist_ok=True)

# Initialize counters for unique filenames and batches
image_counter = 1
mask_counter = 1
batch_number = 1  # Initialize batch number

minmaxscaler = MinMaxScaler()  # Initialize MinMaxScaler

for image_type in ['aug_image', 'aug_mask']:
    image_folder = os.path.join(dataset_root_folder, image_type)
    if not os.path.exists(image_folder):
        continue  # Skip if the folder doesn't exist

    for image_file in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_file)
        #print(image_path)
        if not os.path.isfile(image_path):
            continue  # Skip if it's not a file

        image = cv2.imread(image_path, 1)
        #print(image.shape)
        if image is not None:
            if image_type == 'aug_mask':
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            size_x = (image.shape[1] // image_patch_size) * image_patch_size
            size_y = (image.shape[0] // image_patch_size) * image_patch_size

            image = Image.fromarray(image)
            image = image.crop((0, 0, size_x, size_y))
            image = np.array(image)

            patched_images = patchify(image, (image_patch_size, image_patch_size, 3), step=image_patch_size)
           #print(patched_images.shape[0])
            for i in range(patched_images.shape[0]):
                for j in range(patched_images.shape[1]):
                    if image_type == 'aug_image':
                        individual_patched_image = patched_images[i, j, :, :]
                        #print(individual_patched_image.shape)
                        individual_patched_image = minmaxscaler.fit_transform(
                            individual_patched_image.reshape(-1, individual_patched_image.shape[-1])
                        ).reshape(individual_patched_image.shape)
                        individual_patched_image = individual_patched_image[0]
                        #print(individual_patched_image.shape)

                        # Save the patched image with a unique filename including batch number
                        image_filename = f'patch_{image_counter}.jpg'
                        #print(image_filename)
                        image_save_path = os.path.join(output_images_folder, image_filename)
                        #print(image_save_path)
                        individual_patched_image = (individual_patched_image * 255).astype(np.uint8)
                        cv2.imwrite(image_save_path, individual_patched_image)  # Scale it back to [0, 255]
                        #cv2.imwrite(image_save_path, individual_patched_image)  # Scale it back to [0, 255]
                        # Increment the image counter
                        image_counter += 1

                    elif image_type == 'aug_mask':
                        individual_patched_mask = patched_images[i, j, :, :]
                        individual_patched_mask = individual_patched_mask[0]
                        # Save the patched mask with a unique filename including batch number
                        mask_filename = f'patch_{mask_counter}.png'
                        mask_save_path = os.path.join(output_masks_folder, mask_filename)
                        #print(mask_save_path)
                        cv2.imwrite(mask_save_path, individual_patched_mask )  # Scale it back to [0, 255]

                        # Increment the mask counter
                        mask_counter += 1

    # Increment batch number after processing all images and masks in a batch
#    batch_number += 1
